In [1]:
## IMPORTS ##
import scipy.sparse
import numpy as np
import matplotlib.pyplot as pyplot
import sklearn.metrics as mtc
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MaxAbsScaler
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split

In [7]:
X_tr = scipy.sparse.load_npz('../../Datasets/dataset/X_tr.npz')
X_tst = scipy.sparse.load_npz('../../Datasets/dataset/X_tst.npz')
y_tr = np.load('../../Datasets/dataset/y_tr.npy')
y_tst = np.load('../../Datasets/dataset/y_tst.npy')
# Se binarizan las caracteristicas 
y_tr=MultiLabelBinarizer().fit_transform(y_tr)
y_tst=MultiLabelBinarizer().fit_transform(y_tst)
# Escalado 
scaler = MaxAbsScaler().fit(X_tr)
X_tr=scaler.transform(X_tr)
X_tst=scaler.transform(X_tst)


In [73]:
# REDUCCIÓN DE DIMENSIÓN PRIMERA TÉCNICA
svd = TruncatedSVD(n_components=17, n_iter=10, random_state=42)
X_tr = svd.fit_transform(X_tr)  
X_tst = svd.transform(X_tst)  
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())
print(svd.singular_values_)



ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [26]:
#REDUCCION TAMAÑO TRAINING Y ELIMINAR CARACTERISTICAS INSERVIBLES

columns=np.array([])
for i in range(np.shape(X_tr)[1]):
    if X_tr[:,i].count_nonzero()> 250:
        columns=np.append(columns,i)


In [27]:
#Se guardan las caracteristicas mas importantes para evitar repetir este proceso siempre
np.savetxt('features250.txt',columns,delimiter=',')

print(np.shape(columns))


(4295,)


In [16]:
#Se cargan las caracteristicas guardadas
columns=np.loadtxt('features500.txt',delimiter=',')
X_tr2=X_tr[:,columns]

In [30]:
#Reducir tamaño de muestra para un tunning mas rapido
X_aux, __, y_aux, __ = train_test_split(X_tr2, y_tr, test_size=0.1, random_state=42)


In [8]:
## Clasificador de regresión logística
classif = OneVsRestClassifier(LogisticRegression())
parameters = {
    "estimator__C": [10],
    "estimator__solver": ["liblinear"],
    "estimator__tol": [0.1]
}

model_tunning = GridSearchCV(classif, param_grid=parameters,cv=4)

model_tunning.fit(X_aux, y_aux)

print (model_tunning.best_score_)
print (model_tunning.best_params_)


NameError: name 'X_aux' is not defined

In [9]:
classif = OneVsRestClassifier(LogisticRegression())
parameters = {
    "estimator__C": [1],
    "estimator__solver": ["liblinear"],
    "estimator__tol": [0.01],
    "estimator__class_weight":[None]
}

model_tunning = GridSearchCV(classif, param_grid=parameters,cv=4)

model_tunning.fit(X_tr, y_tr)

print (model_tunning.best_score_)
print (model_tunning.best_params_)

0.37310668554215587
{'estimator__C': 1, 'estimator__class_weight': None, 'estimator__solver': 'liblinear', 'estimator__tol': 0.01}


In [74]:
#classif=OneVsRestClassifier(LogisticRegression(C=0.6,tol=0.1,solver='liblinear'))
#classif.fit(X_tr,y_tr)



In [10]:
y_pred=model_tunning.predict(X_tst)
accuracy=mtc.accuracy_score(y_tst,y_pred)
hamming=mtc.hamming_loss(y_tst,y_pred)
print(accuracy)
print(hamming)


0.35991016282987087
0.03130643276628678


In [ ]:
for i in range(len(y_pred)):
    print ("Accuracy:" mtc.accuracy_score(y_tst[:,i],y_pred[:,i]))

In [ ]:
#RBM -- SE IMPLEMENTARÁ CON UN PIPELINE PERO AÚN FALTA ENTENDER MEJOR EL FUNCIONAMIENTO DE BERNOULLIRBM -- 
#-- NO EJECUTAR-- #
logistic = OneVsRestClassifier(LogisticRegression())
rbm = BernoulliRBM(random_state=0, verbose=True)
rbm_features_classifier = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])
rbm.learning_rate = 0.06
rbm.n_iter = 20
rbm.n_components = 100
rbm_features_classifier.fit(X_tr, y_tr)
Y_pred2 = rbm_features_classifier.predict(X_tst)
